## Here's to my first competition ever!

#### My semester just started again, so I might not be able to remain as (super) active I recently was on Kaggle, but it sure feels amazing to finally participate in a competition.

#### This notebook took a lot out of me. While Travis Kelce has been on my radar since he started dating Taylor Swift, I have absolutely **zero** knowledge of football, and analysing this data taught me more about it than any TV show or internet search did. 

### This notebook contains a few fun features:
#### 1. Tackle animations on a graphed football field! (felt really cool completing that) 
#### 2. A classification model with an accuracy of 88.47% classifying by the tackle success.
#### 3. Data processing and visualisations that will explain how tackles were affected by different metrics.

##### Hope you have fun reading this notebook, let me know of any feedbacks!


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nfl-big-data-bowl-2024/players.csv
/kaggle/input/nfl-big-data-bowl-2024/tracking_week_7.csv
/kaggle/input/nfl-big-data-bowl-2024/tracking_week_9.csv
/kaggle/input/nfl-big-data-bowl-2024/tracking_week_6.csv
/kaggle/input/nfl-big-data-bowl-2024/games.csv
/kaggle/input/nfl-big-data-bowl-2024/tracking_week_8.csv
/kaggle/input/nfl-big-data-bowl-2024/tracking_week_4.csv
/kaggle/input/nfl-big-data-bowl-2024/tracking_week_3.csv
/kaggle/input/nfl-big-data-bowl-2024/tackles.csv
/kaggle/input/nfl-big-data-bowl-2024/tracking_week_5.csv
/kaggle/input/nfl-big-data-bowl-2024/tracking_week_1.csv
/kaggle/input/nfl-big-data-bowl-2024/plays.csv
/kaggle/input/nfl-big-data-bowl-2024/tracking_week_2.csv


## Loading datasets

In [2]:
games = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2024/games.csv')
games.head()

,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,homeFinalScore,visitorFinalScore
0,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31
1,2022091100,2022,1,09/11/2022,13:00:00,ATL,NO,26,27
2,2022091101,2022,1,09/11/2022,13:00:00,CAR,CLE,24,26
3,2022091102,2022,1,09/11/2022,13:00:00,CHI,SF,19,10
4,2022091103,2022,1,09/11/2022,13:00:00,CIN,PIT,20,23


In [3]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   gameId             136 non-null    int64 
 1   season             136 non-null    int64 
 2   week               136 non-null    int64 
 3   gameDate           136 non-null    object
 4   gameTimeEastern    136 non-null    object
 5   homeTeamAbbr       136 non-null    object
 6   visitorTeamAbbr    136 non-null    object
 7   homeFinalScore     136 non-null    int64 
 8   visitorFinalScore  136 non-null    int64 
dtypes: int64(5), object(4)
memory usage: 9.7+ KB


In [4]:
players = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2024/players.csv')
players.head()

,nflId,height,weight,birthDate,collegeName,position,displayName
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady
1,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters
2,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers
3,30842,6-6,267,1984-05-19,UCLA,TE,Marcedes Lewis
4,33084,6-4,217,1985-05-17,Boston College,QB,Matt Ryan


In [5]:
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1683 entries, 0 to 1682
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   nflId        1683 non-null   int64 
 1   height       1683 non-null   object
 2   weight       1683 non-null   int64 
 3   birthDate    1204 non-null   object
 4   collegeName  1683 non-null   object
 5   position     1683 non-null   object
 6   displayName  1683 non-null   object
dtypes: int64(2), object(5)
memory usage: 92.2+ KB


In [6]:
plays = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2024/plays.csv')
plays.head()

,gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,...,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2
0,2022100908,3537,48723,Parker Hesse,(7:52) (Shotgun) M.Mariota pass short middle t...,4,1,10,ATL,TB,...,0.976785,0.023215,-0.006110,0.006110,2.360609,0.981955,NaN,NaN,NaN,NaN
1,2022091103,3126,52457,Chase Claypool,(7:38) (Shotgun) C.Claypool right end to PIT 3...,4,1,10,PIT,CIN,...,0.160485,0.839515,-0.010865,0.010865,1.733344,-0.263424,NaN,NaN,NaN,NaN
2,2022091111,1148,42547,Darren Waller,(8:57) D.Carr pass short middle to D.Waller to...,2,2,5,LV,LAC,...,0.756661,0.243339,-0.037409,0.037409,1.312855,1.133666,NaN,NaN,NaN,NaN
3,2022100212,2007,46461,Mike Boone,(13:12) M.Boone left tackle to DEN 44 for 7 ya...,3,2,10,DEN,LV,...,0.620552,0.379448,-0.002451,0.002451,1.641006,-0.043580,NaN,NaN,NaN,NaN
4,2022091900,1372,47857,Devin Singletary,(8:33) D.Singletary right guard to TEN 32 for ...,2,1,10,BUF,TEN,...,0.836290,0.163710,0.001053,-0.001053,3.686428,-0.167903,NaN,NaN,NaN,NaN


In [7]:
plays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12486 entries, 0 to 12485
Data columns (total 35 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   gameId                            12486 non-null  int64  
 1   playId                            12486 non-null  int64  
 2   ballCarrierId                     12486 non-null  int64  
 3   ballCarrierDisplayName            12486 non-null  object 
 4   playDescription                   12486 non-null  object 
 5   quarter                           12486 non-null  int64  
 6   down                              12486 non-null  int64  
 7   yardsToGo                         12486 non-null  int64  
 8   possessionTeam                    12486 non-null  object 
 9   defensiveTeam                     12486 non-null  object 
 10  yardlineSide                      12319 non-null  object 
 11  yardlineNumber                    12486 non-null  int64  
 12  game

In [8]:
tackles = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2024/tackles.csv')
tackles.head()

,gameId,playId,nflId,tackle,assist,forcedFumble,pff_missedTackle
0,2022090800,101,42816,1,0,0,0
1,2022090800,393,46232,1,0,0,0
2,2022090800,486,40166,1,0,0,0
3,2022090800,646,47939,1,0,0,0
4,2022090800,818,40107,1,0,0,0


In [9]:
tackles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17426 entries, 0 to 17425
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   gameId            17426 non-null  int64
 1   playId            17426 non-null  int64
 2   nflId             17426 non-null  int64
 3   tackle            17426 non-null  int64
 4   assist            17426 non-null  int64
 5   forcedFumble      17426 non-null  int64
 6   pff_missedTackle  17426 non-null  int64
dtypes: int64(7)
memory usage: 953.1 KB


In [10]:
track1 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2024/tracking_week_1.csv')
track1.head()

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
0,2022090800,56,35472.0,Rodger Saffold,1,2022-09-08 20:24:05.200000,76.0,BUF,left,88.37,27.27,1.62,1.15,0.16,231.74,147.90,NaN
1,2022090800,56,35472.0,Rodger Saffold,2,2022-09-08 20:24:05.299999,76.0,BUF,left,88.47,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived
2,2022090800,56,35472.0,Rodger Saffold,3,2022-09-08 20:24:05.400000,76.0,BUF,left,88.56,27.01,1.57,0.49,0.15,230.98,147.05,NaN
3,2022090800,56,35472.0,Rodger Saffold,4,2022-09-08 20:24:05.500000,76.0,BUF,left,88.64,26.90,1.44,0.89,0.14,232.38,145.42,NaN
4,2022090800,56,35472.0,Rodger Saffold,5,2022-09-08 20:24:05.599999,76.0,BUF,left,88.72,26.80,1.29,1.24,0.13,233.36,141.95,NaN


In [11]:
track1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1407439 entries, 0 to 1407438
Data columns (total 17 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   gameId         1407439 non-null  int64  
 1   playId         1407439 non-null  int64  
 2   nflId          1346246 non-null  float64
 3   displayName    1407439 non-null  object 
 4   frameId        1407439 non-null  int64  
 5   time           1407439 non-null  object 
 6   jerseyNumber   1346246 non-null  float64
 7   club           1407439 non-null  object 
 8   playDirection  1407439 non-null  object 
 9   x              1407439 non-null  float64
 10  y              1407439 non-null  float64
 11  s              1407439 non-null  float64
 12  a              1407439 non-null  float64
 13  dis            1407439 non-null  float64
 14  o              1346397 non-null  float64
 15  dir            1346397 non-null  float64
 16  event          130268 non-null   object 
dtypes: float

## Data Processing

#### To understand the factors that influence the success tackle rate,let's merge all data into one final dataframe

#### The merging will be performed on the common columns as coded below.

In [12]:
df_1 = pd.merge(tackles, players, on = 'nflId')

df_2 = pd.merge(df_1, plays, on = ['gameId', 'playId'])

df = pd.merge(df_2, games, on = 'gameId')

df.head()

,gameId,playId,nflId,tackle,assist,forcedFumble,pff_missedTackle,height,weight,birthDate,...,foulNFLId1,foulNFLId2,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,homeFinalScore,visitorFinalScore
0,2022090800,101,42816,1,0,0,0,5-11,184,1991-08-29,...,NaN,NaN,2022,1,09/08/2022,20:20:00,LA,BUF,10,31
1,2022090800,1102,42816,1,0,0,0,5-11,184,1991-08-29,...,NaN,NaN,2022,1,09/08/2022,20:20:00,LA,BUF,10,31
2,2022090800,1102,52492,0,0,0,1,6-5,252,1998-08-25,...,NaN,NaN,2022,1,09/08/2022,20:20:00,LA,BUF,10,31
3,2022090800,2043,42816,1,0,0,0,5-11,184,1991-08-29,...,NaN,NaN,2022,1,09/08/2022,20:20:00,LA,BUF,10,31
4,2022090800,2527,42816,1,0,0,0,5-11,184,1991-08-29,...,NaN,NaN,2022,1,09/08/2022,20:20:00,LA,BUF,10,31


In [13]:
df['visitorTeamAbbr'].unique()

array(['BUF', 'ATL', 'LA', 'SF', 'GB', 'TEN', 'CAR', 'DAL', 'PIT', 'NO',
       'TB', 'CIN', 'BAL', 'LV', 'SEA', 'MIN', 'CLE', 'LAC', 'NYJ', 'NE',
       'ARI', 'KC', 'HOU', 'CHI', 'WAS', 'MIA', 'PHI', 'DET', 'JAX',
       'IND', 'NYG', 'DEN'], dtype=object)

In [14]:
df.select_dtypes(include=['object']).columns

Index(['height', 'birthDate', 'collegeName', 'position', 'displayName',
       'ballCarrierDisplayName', 'playDescription', 'possessionTeam',
       'defensiveTeam', 'yardlineSide', 'gameClock', 'passResult',
       'playNullifiedByPenalty', 'offenseFormation', 'foulName1', 'foulName2',
       'gameDate', 'gameTimeEastern', 'homeTeamAbbr', 'visitorTeamAbbr'],
      dtype='object')

In [15]:
df['playNullifiedByPenalty'].unique()

array(['N', 'Y'], dtype=object)

In [16]:
df['passResult'].unique()

array([nan, 'C', 'R'], dtype=object)

In [17]:
df.describe()

,gameId,playId,nflId,tackle,assist,forcedFumble,pff_missedTackle,weight,ballCarrierId,quarter,...,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulNFLId1,foulNFLId2,season,week,homeFinalScore,visitorFinalScore
count,1.742600e+04,17426.000000,17426.000000,17426.000000,17426.000000,17426.000000,17426.000000,17426.000000,17426.000000,17426.000000,...,17426.000000,17426.000000,17426.000000,17424.000000,761.000000,31.000000,17426.0,17426.000000,17426.000000,17426.000000
mean,2.022099e+09,1982.974578,47602.719442,0.569207,0.315276,0.005681,0.119936,232.930391,48188.776770,2.550212,...,-0.000067,0.000067,2.215160,0.242920,47765.601840,49575.258065,2022.0,4.874670,22.704178,21.055664
std,5.978775e+03,1170.990596,4846.585868,0.495201,0.464639,0.075161,0.324896,39.927658,4606.422151,1.132602,...,0.034437,0.034437,1.585774,1.136264,4987.646861,3570.559657,0.0,2.561432,8.580195,9.530385
min,2.022091e+09,54.000000,33131.000000,0.000000,0.000000,0.000000,0.000000,170.000000,25511.000000,1.000000,...,-0.521181,-0.410824,-2.366872,-9.986150,33107.000000,43586.000000,2022.0,1.000000,3.000000,0.000000
25%,2.022093e+09,952.000000,43797.000000,0.000000,0.000000,0.000000,0.000000,200.000000,44874.000000,2.000000,...,-0.010758,-0.010872,1.046297,-0.445271,43503.000000,46527.000000,2022.0,3.000000,17.000000,15.000000
50%,2.022101e+09,1991.000000,46669.000000,1.000000,0.000000,0.000000,0.000000,227.000000,47807.000000,3.000000,...,0.000007,-0.000007,1.992672,0.063363,47791.000000,47996.000000,2022.0,5.000000,22.000000,20.000000
75%,2.022102e+09,2981.750000,52546.000000,1.000000,1.000000,0.000000,0.000000,252.000000,52754.750000,4.000000,...,0.010872,0.010758,3.360313,0.818275,53063.000000,53046.000000,2022.0,7.000000,27.000000,27.000000
max,2.022111e+09,5096.000000,55241.000000,1.000000,1.000000,1.000000,1.000000,379.000000,55158.000000,5.000000,...,0.410824,0.521181,6.556614,7.130048,55157.000000,54650.000000,2022.0,9.000000,49.000000,48.000000


#### From the observations, we can convert a few columns from string to integer type by encoding or conversion.

In [18]:
#Converting height in feet-inchs format to inches

def convert_height_to_inches(height):
    feet, inches = map(int, height.split('-'))
    # Convert feet to inches and sum with inches
    return feet * 12 + inches

df['height'] = df['height'].apply(convert_height_to_inches)

In [19]:
df['playNullifiedByPenalty'] = df['playNullifiedByPenalty'].replace({'N': 0, 'Y': 1})
df['passResult'] = df['passResult'].replace({'C': 0, 'R': 1})
df = df.drop('season', axis=1) #as all values equal to 2022

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17426 entries, 0 to 17425
Data columns (total 53 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   gameId                            17426 non-null  int64  
 1   playId                            17426 non-null  int64  
 2   nflId                             17426 non-null  int64  
 3   tackle                            17426 non-null  int64  
 4   assist                            17426 non-null  int64  
 5   forcedFumble                      17426 non-null  int64  
 6   pff_missedTackle                  17426 non-null  int64  
 7   height                            17426 non-null  int64  
 8   weight                            17426 non-null  int64  
 9   birthDate                         13526 non-null  object 
 10  collegeName                       17426 non-null  object 
 11  position                          17426 non-null  object 
 12  disp

In [21]:
import plotly.express as px

In [22]:
# Create a correlation matrix
df_corr = df[['tackle','assist', 'forcedFumble', 'pff_missedTackle', 'height', 'weight', 'position', 'playResult', 'passResult', 'passLength', 'defendersInTheBox', 'passProbability']]

df_corr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17426 entries, 0 to 17425
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tackle             17426 non-null  int64  
 1   assist             17426 non-null  int64  
 2   forcedFumble       17426 non-null  int64  
 3   pff_missedTackle   17426 non-null  int64  
 4   height             17426 non-null  int64  
 5   weight             17426 non-null  int64  
 6   position           17426 non-null  object 
 7   playResult         17426 non-null  int64  
 8   passResult         7606 non-null   float64
 9   passLength         7018 non-null   float64
 10  defendersInTheBox  17421 non-null  float64
 11  passProbability    16994 non-null  float64
dtypes: float64(4), int64(7), object(1)
memory usage: 1.6+ MB


In [23]:
#Converting position from categorical to numerical dummy values
df['position'].unique()

array(['CB', 'OLB', 'DE', 'DT', 'FS', 'ILB', 'SS', 'NT', 'MLB', 'DB'],
      dtype=object)

In [24]:
df_corr = pd.get_dummies(df_corr, columns=['position'], prefix='')

In [25]:
df_corr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17426 entries, 0 to 17425
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tackle             17426 non-null  int64  
 1   assist             17426 non-null  int64  
 2   forcedFumble       17426 non-null  int64  
 3   pff_missedTackle   17426 non-null  int64  
 4   height             17426 non-null  int64  
 5   weight             17426 non-null  int64  
 6   playResult         17426 non-null  int64  
 7   passResult         7606 non-null   float64
 8   passLength         7018 non-null   float64
 9   defendersInTheBox  17421 non-null  float64
 10  passProbability    16994 non-null  float64
 11  _CB                17426 non-null  bool   
 12  _DB                17426 non-null  bool   
 13  _DE                17426 non-null  bool   
 14  _DT                17426 non-null  bool   
 15  _FS                17426 non-null  bool   
 16  _ILB               174

In [26]:
correlation_matrix = df_corr.corr()

fig = px.imshow(correlation_matrix,
                labels=dict(color="Correlation"),
                x=correlation_matrix.index,
                y=correlation_matrix.columns,
                color_continuous_scale='Plasma',
                title='Correlation Heatmap of Features')

# Show the plot
fig.show()

In [27]:
correlation_matrix = df_corr.corr()
tackle_correlation = correlation_matrix['tackle'].sort_values(ascending=False)
tackle_correlation

tackle               1.000000
passLength           0.154934
_CB                  0.106028
passProbability      0.099034
passResult           0.031919
_FS                  0.030577
forcedFumble         0.028754
_SS                  0.022230
playResult           0.012940
_DB                  0.004103
_MLB                -0.004003
_ILB                -0.007719
_OLB                -0.036505
_NT                 -0.038472
_DE                 -0.055614
defendersInTheBox   -0.065325
_DT                 -0.077428
height              -0.089990
weight              -0.134979
pff_missedTackle    -0.411859
assist              -0.779989
Name: tackle, dtype: float64

In [28]:
tackle_correlation

tackle               1.000000
passLength           0.154934
_CB                  0.106028
passProbability      0.099034
passResult           0.031919
_FS                  0.030577
forcedFumble         0.028754
_SS                  0.022230
playResult           0.012940
_DB                  0.004103
_MLB                -0.004003
_ILB                -0.007719
_OLB                -0.036505
_NT                 -0.038472
_DE                 -0.055614
defendersInTheBox   -0.065325
_DT                 -0.077428
height              -0.089990
weight              -0.134979
pff_missedTackle    -0.411859
assist              -0.779989
Name: tackle, dtype: float64

In [29]:
# Create a horizontal bar chart for correlation with tackle 
fig1 = px.bar(x=tackle_correlation.values,
             y=tackle_correlation.index,
             orientation='h',
             labels={'x': 'Correlation', 'y': 'Columns'},
             title='Correlation with Tackles')
fig1.show()

#### Columns having lower correlation values are those from height to passProbability. We can eliminate those columns.

In [30]:
tackle_correlation = tackle_correlation[(tackle_correlation>0.1)|(tackle_correlation<-0.1)]

In [31]:
# Create a horizontal bar chart for correlation with tackle 
fig1 = px.bar(x=tackle_correlation.values,
             y=tackle_correlation.index,
             orientation='h',
             labels={'x': 'Correlation', 'y': 'Columns'},
              color = tackle_correlation.values,
             title='Correlation with Tackles')
fig1.show()

### Let's observe and interpret:
#### 1. Players with assistance are least likely to make successful tackles.
#### 2. The higher the weight of a player, the less likely is the tackle.
#### 3. Cornerback is the only position that has a positive correlation with the successful tackle rate.

## Data Visualisation : Animations and More

### Let's try and create a dummy of a football field:

#### We will create a function that will take the input of the dataframe, the x and y coordinates of the players, the speed of that play's player, and the display name of the player. These inputs will help us create a proper classification of all plays and players on the visualisation.

In [32]:
import plotly.graph_objects as go

def create_field(df, x, y, s, p):
    fig = px.scatter(df, x=x, y=y, size=s, labels={'size' : 'Speed (yards/second)'}, color=p)
    # Add the center circle as a marker
    fig.add_trace(go.Scatter(
        x=[60],
        y=[26.65],
        mode='markers',
        marker=dict(size=100, color='lightgreen'),
        showlegend=False,
    ))

    fig.update_layout(
        title='Football Field with Player Positions',
        xaxis=dict(title='Player Position (yards)'),
        yaxis=dict(title='Player Position (yards)'),
        shapes=[
            # Left Goal Area
            dict(type='rect', x0=0, x1=6, y0=20.16, y1=33.13, line=dict(color='green')),
            # Right Goal Area
            dict(type='rect', x0=114, x1=120, y0=20.16, y1=33.13, line=dict(color='green')),
            # Center Circle
            dict(type='circle', xref='x', yref='y', x0=59, y0=26.65, x1=61, y1=26.65, line=dict(color='green')),
            # Halfway Line
            dict(type='line', x0=60, x1=60, y0=0, y1=53.3, line=dict(color='green')),
        ],
    )

    # Show the plot
    fig.show()


In [33]:
track1['event'].unique()

array([nan, 'pass_arrived', 'pass_outcome_caught', 'tackle', 'run',
       'first_contact', 'ball_snap', 'handoff', 'touchdown',
       'out_of_bounds', 'man_in_motion', 'fumble', 'play_action',
       'pass_forward', 'lateral', 'autoevent_passforward',
       'autoevent_passinterrupted', 'line_set', 'qb_slide', 'shift',
       'run_pass_option', 'qb_sack', 'pass_shovel', 'autoevent_ballsnap',
       'snap_direct', 'fumble_defense_recovered',
       'fumble_offense_recovered'], dtype=object)

#### Let's try a sample plotting:

In [34]:
create_field(track1, track1['x'].iloc[:100], track1['y'].iloc[:100], track1['s'].iloc[:100], track1['displayName'].iloc[:100])

#### This looks great! We can see a static visualisation of the players moving across the football field varied by their speed.

#### Let's isolate those play sessions that ended up in a tackle event.

In [35]:
track1.loc[track1['event']=='tackle', 'playId'].nunique()

1012

#### There are 1012 plays that ended in a tackle. Let's try to visualise these tackles on the basis of the number of records.

In [36]:
def animate_field(df,n):
    # Filter DataFrame for sessions ending in a tackle event
    df = df.iloc[:n]
    tackle_sessions = df[df['event'] == 'tackle']
    x = df['x'].iloc[:n]
    y = df['y'].iloc[:n]
    s = df['s'].iloc[:n]
    f = df['frameId'].iloc[:n]
    d = df['displayName'].iloc[:n]
    # Create a scatter plot for each playId ending in a tackle event
    for playId, play_df in tackle_sessions.groupby(['gameId','playId']):
        fig = px.scatter(play_df, x=x, y=y, size=s, animation_frame=f, color=d,
                         labels={'size': 'Speed (yards/second)'})
        
        fig.add_trace(go.Scatter(
        x=[60],
        y=[26.65],
        mode='markers',
        marker=dict(size=100, color='lightgreen'),
        showlegend=False,
    ))
        # football field layout
        fig.update_layout(
            title=f'Football Field - Game ID, Play ID: {playId} (Tackle Event)',
            xaxis=dict(title='Player Position (yards)'),
            yaxis=dict(title='Player Position (yards)'),
            shapes=[
                # Left Goal Area
                dict(type='rect', x0=0, x1=6, y0=20.16, y1=33.13, line=dict(color='black')),
                # Right Goal Area
                dict(type='rect', x0=114, x1=120, y0=20.16, y1=33.13, line=dict(color='black')),
                # Center Circle
                dict(type='circle', xref='x', yref='y', x0=57, y0=26.65, x1=63, y1=26.65, line=dict(color='black')),
                # Halfway Line
                dict(type='line', x0=60, x1=60, y0=0, y1=53.3, line=dict(color='black')),
            ]
        )

        # Show the plot
        fig.show()

In [37]:
animate_field(track1,100)


### An upgrade to the previous visualisation! We have created a function to animate the play that ended up in a tackle!

### For a more comprehensive understanding of how successful tackles work, let's plot a few graphs:
By our understanding and using the correlation matrix, we can conclude that a few metrics matter in a successful tackle:
1. Height and Weight (hence, tolerance of physical attack and strength of collision)
2. yardsToGo
3. down
4. passLength
5. birthDate

Height:

In [38]:
fig = px.histogram(df, x='height', color='tackle', title='Distribution of Tackles by Player Height')

# Update layout for better visualization
fig.update_layout(
    bargroupgap=0.1,
    xaxis=dict(title='Player Height'),
    yaxis=dict(title='Count of Tackles'),
)

# Show the plot
fig.show()


Weight:

In [39]:
fig_weight_box = px.box(df, x='tackle', y='weight', color='tackle', labels={'weight': 'Player Weight', 'tackle': 'Tackle'},
                        title='Box Plot: Tackle vs Player Weight')
fig_weight_box.update_layout(xaxis=dict(title='Tackle'), yaxis=dict(title='Player Weight'))
fig_weight_box.show()

Pass Length:

In [40]:
figpl = px.histogram(df, x='passLength', color='tackle', barmode='group', labels={'passLength': 'Pass Length', 'tackle': 'Tackle'},
                             title='Bar Plot: Tackle vs Pass Length')
figpl.update_layout(xaxis=dict(title='Pass Length'), yaxis=dict(title='Count'))
figpl.show()

In [41]:
df['downToGo'] = df['yardsToGo']*df['down']
figdown = px.histogram(df, x='downToGo', color='tackle', labels={'passLength': 'Pass Length', 'tackle': 'Tackle'},
                             title='Bar Plot: Yards to Go (Down) vs Tackle')
figdown.update_layout(xaxis=dict(title='Down to Go'), yaxis=dict(title='Count'))

### Let's create a feature dataset for predicting our tackle success. From the correlation matrix, we have inferred that only a few columns directly correlate to the tackle success rate.

We will use the following metrics in our classification model:

In [42]:
comp = df[['tackle', 'assist', 'passLength', 'height', 'weight', 'downToGo', 'quarter', 'forcedFumble']]

We used these metrics due to the following reasons:
1. **assist, passLength** were correlated in the heatmap.
2. The **height and weight** may determine the strength of the player, hence affect tackles.
3. The **quarter** is a good indicative of the time of the game passed.
4. The **downToGo** is a measure of how the team will plan its next offense.
5. The **forcedFumble** is a measure of the team's and the player's offensive abilities.

In [43]:
comp

,tackle,assist,passLength,height,weight,downToGo,quarter,forcedFumble
0,1,0,NaN,71,184,10,1,0
1,1,0,-3.0,71,184,12,2,0
2,0,0,-3.0,77,252,12,2,0
3,1,0,NaN,71,184,10,3,0
4,1,0,9.0,71,184,20,3,0
...,...,...,...,...,...,...,...,...
17421,1,0,NaN,76,265,12,4,0
17422,1,0,NaN,76,265,10,4,0
17423,1,0,NaN,73,198,10,1,1
17424,1,0,4.0,73,198,18,1,0


## Let's build our classification model!

#### We will use a Support Vector Machine model.

In [44]:
comp=comp.fillna(method='bfill')

In [45]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

In [46]:
# Splitting the dataset into training, validation, and test sets
train, test = train_test_split(comp, test_size=0.8, random_state=42)

X_train = pd.get_dummies(train.drop('tackle', axis=1), drop_first=True)
y_train = train['tackle']

X_test = pd.get_dummies(test.drop('tackle', axis=1), drop_first=True)
y_test = test['tackle']

# Standardize the numerical features
# Note: Only standardize numerical columns. For simplicity, I'm assuming all columns are now either numerical or one-hot encoded
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Now, the dataset should be ready for training and evaluating a model

In [47]:
# Create a Support Vector Classifier
svc = SVC(kernel='linear', C=1.0, random_state=42)

# Train the model
svc.fit(X_train, y_train)

# Make predictions
y_pred = svc.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")

Model Accuracy: 0.884728498672979


## Our Support Vector Classifier works with an accuracy of 88.47%

#### This gives us the opportunity to use it for real-world deployment as well. The accuracy is evident of the fact that with the metrics we chose, the tackle success rate is most affected.